# Redes Neurais Recorrentes

Long-Short Term Memory (LSTM) são um tipo de rede neural, mais especificamente um tipo de rede neural recorrente.

Redes neurais recorrentes possuem uma característica distinta de redes neurais feedforward. Ao invés da informação seguir um fluxo contínuo sempre em um direção (usualmente para "frente"), as RNNs passam a informação também de volta ("trás"). Isso permite que essas simulem uma memória, sendo capazes de lidar melhor com problemas que variam com o tempo, como é o nosso caso.

In [41]:
import torch
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd

## Tensores

É a principal estrutura de dado do framework. Ela é essencialmente um array numpy de 3 dimensões. Entretanto, com tensores é possível realizar operações diretamente na GPU aumentando a eficiência dos algoritmos.

In [2]:
torch.rand(2, 3, 2)

tensor([[[0.1541, 0.3130],
         [0.1192, 0.3687],
         [0.1044, 0.6758]],

        [[0.8448, 0.7562],
         [0.7207, 0.6760],
         [0.0249, 0.2527]]])

Temos suporte direto a operações aritméticas assim como no numPy. Podemos realizar adições (`+`), subtrações (`-`), multiplicação por escalar (`*`), transposição (`T` ou `transp()`), produto de hadarmard (`*`), produto interno (`@`), entre outros.

In [3]:
A = torch.ones(2, 3, 2, dtype=torch.int) * 2
I = torch.eye(3, 2, dtype=torch.int)

In [4]:
A

tensor([[[2, 2],
         [2, 2],
         [2, 2]],

        [[2, 2],
         [2, 2],
         [2, 2]]], dtype=torch.int32)

In [5]:
I

tensor([[1, 0],
        [0, 1],
        [0, 0]], dtype=torch.int32)

In [6]:
A + I

tensor([[[3, 2],
         [2, 3],
         [2, 2]],

        [[3, 2],
         [2, 3],
         [2, 2]]], dtype=torch.int32)

In [7]:
A * I

tensor([[[2, 0],
         [0, 2],
         [0, 0]],

        [[2, 0],
         [0, 2],
         [0, 0]]], dtype=torch.int32)

In [8]:
# A * I == A @ I

In [9]:
A @ torch.eye(2, dtype=torch.int)

tensor([[[2, 2],
         [2, 2],
         [2, 2]],

        [[2, 2],
         [2, 2],
         [2, 2]]], dtype=torch.int32)

In [10]:
np.pi * A

tensor([[[6.2832, 6.2832],
         [6.2832, 6.2832],
         [6.2832, 6.2832]],

        [[6.2832, 6.2832],
         [6.2832, 6.2832],
         [6.2832, 6.2832]]])

In [11]:
rng = torch.rand(1, 3, 2)
A + rng

tensor([[[2.6189, 2.1177],
         [2.6972, 2.6214],
         [2.3023, 2.3396]],

        [[2.6189, 2.1177],
         [2.6972, 2.6214],
         [2.3023, 2.3396]]])

In [12]:
A.view(-1, 4)

tensor([[2, 2, 2, 2],
        [2, 2, 2, 2],
        [2, 2, 2, 2]], dtype=torch.int32)

A execução na GPU não ocorre de forma automática. Primeiro, podemos verificar se o dispositivo é suportado pelo PyTorch.

In [13]:
torch.cuda.is_available()

/home/thalesaguiar/.pyenv/versions/3.8.2/envs/fakenews.env/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


False

Em caso positivo, podemos especificar quais tensores terão suas operações executadas na placa gráfica, definir a execução de tudo por padrão.

In [14]:
# gpu = torch.device("cuda")
# A.to(gpu)
# cuda_tensor = torch.tensor([3, 4, 5], device=gpu)

In [16]:
a_view = A.view(-1, 3)
a_view

tensor([[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]], dtype=torch.int32)

In [17]:
A[0, 0, 0] = 99999
a_view

tensor([[99999,     2,     2],
        [    2,     2,     2],
        [    2,     2,     2],
        [    2,     2,     2]], dtype=torch.int32)

In [18]:
a_view[0, 2] = 55
A

tensor([[[99999,     2],
         [   55,     2],
         [    2,     2]],

        [[    2,     2],
         [    2,     2],
         [    2,     2]]], dtype=torch.int32)

## Rede Neural

O PyTorch permite criar modelos neurais com bastante facilidade e praticidade. Tanto a criação de modelos customizáveis e a grande quantidade de modelos prontos disponíveis é bastante simples. Vamos gerar um conjunto de dados aleatórios para experimentar as funções do framework.

In [19]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

X, Y = load_boston(return_X_y=True)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3,
                                                shuffle=True)
Xdf = pd.DataFrame(X)
Xdf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


Transformando os dados para tensores, temos

In [20]:
Xtrain = torch.tensor(Xtrain, dtype=torch.float)
Xtest = torch.tensor(Xtest, dtype=torch.float)
Ytrain = torch.tensor(Ytrain, dtype=torch.float).view(-1, 1)
Ytest = torch.tensor(Ytest, dtype=torch.float).view(-1, 1)

Por exemplo, abaixo podemos criar uma rede neural simples apenas usando os tensores.

In [21]:
weights = torch.rand(1, 13, requires_grad=True)
weights

tensor([[0.7587, 0.7873, 0.2541, 0.4459, 0.9125, 0.3273, 0.0721, 0.7155, 0.8789,
         0.9669, 0.1373, 0.5753, 0.1432]], requires_grad=True)

In [22]:
bias = torch.rand(1, 1, requires_grad=True)
bias

tensor([[0.5239]], requires_grad=True)

O parâmetro `requires_grad` diz ao PyTorch que esse tensor deve ser levado em consideração quando algum otimizador for utilizado. Ou seja, no momento que fazemos a propagação do erro, esse tensor será atualizado.

Então, temos o seguinte problema:

$$y = D \times W + b$$

Onde $D$ é nossa base (apenas *features*), $W$ são os pesos, $b$ é um bias representando possíveis ruídos/erros obtidos das aproximações e $y$ são nossos valores esperados. Ou seja

$$\hat{y} = D \times W^{'} + b^{'}$$

Sendo $\hat{y}$ um valor próximo o suficiente de $y$, assim como $W^{'}$ e $b^{'}$.

In [23]:
yhat = Xtrain @ weights.T + bias
yhat[:3]

tensor([[707.5792],
        [512.8845],
        [516.8206]], grad_fn=<SliceBackward>)

In [24]:
def model(X):
    return X @ weights.T + bias

Agora, precisamos definir a função objetivo. Vamos usar um simples Mean Squared Error.

In [25]:
sqr_diff = (Ytrain - yhat) ** 2
torch.sum(sqr_diff) / sqr_diff.numel()

tensor(402191., grad_fn=<DivBackward0>)

A operação `numel()` retorna a quantidade de elementos do tensor. Vamos criar uma função para calcular o erro

In [26]:
def mse(preds, real):
    diff = (real - preds) ** 2
    return torch.sum(diff) / diff.numel()

Para descobrir o quanto atualizar cada peso, vamos aplicar o *gradient descent*. 

In [28]:
loss = mse(Ytrain, yhat)
loss.backward()
weights

tensor([[0.7587, 0.7873, 0.2541, 0.4459, 0.9125, 0.3273, 0.0721, 0.7155, 0.8789,
         0.9669, 0.1373, 0.5753, 0.1432]], requires_grad=True)

Note que os valores do tensor não foram atualizados após executar o backpropagation. Isso é porque os valores do gradiente são armazenados no atributo `grad` de cada tensor

In [29]:
weights.grad

tensor([[5.8191e+03, 1.2228e+04, 1.4878e+04, 6.5590e+01, 7.0096e+02, 7.6143e+03,
         8.7374e+04, 4.3591e+03, 1.3827e+04, 5.4740e+05, 2.2943e+04, 4.3890e+05,
         1.6771e+04]])

Agora, podemos atualizar o tensor com o seu gradiente. Aqui devemos tomar cuidado, pois o PyTorch controla e memoriza todas as operações realizadas nos tensores com `require_grad=True` para utilizar no cálculo dos gradientes. Para atualizar os parâmetros, não queremos que essa operação seja gravada. Para evitar esse comortamento, podemos usar o operador de contexto `with` com a função `torch.no_grad()`.

In [33]:
# hyperparameters
weights.grad.zero_()
bias.grad.zero_()

nepochs = 250
lrate = 1e-15

for epoch in range(nepochs):
    predictions = model(Xtrain)
    loss = mse(predictions, Ytrain)
    loss.backward()
    if epoch % 10 == 0:
        print(f'{epoch:3}° {loss}')
    with torch.no_grad():
        weights -= weights.grad * lrate
        bias -= bias.grad * lrate
        weights.grad.zero_()
        bias.grad.zero_()

  0° nan
 10° nan
 20° nan
 30° nan
 40° nan
 50° nan
 60° nan
 70° nan
 80° nan
 90° nan
100° nan
110° nan
120° nan
130° nan
140° nan
150° nan
160° nan
170° nan
180° nan
190° nan
200° nan
210° nan
220° nan
230° nan
240° nan


In [32]:
pred = model(Xtest)
loss = mse(pred, Ytest)
loss

tensor(nan, grad_fn=<DivBackward0>)

In [36]:
(Ytest - pred)[:3]

tensor([[nan],
        [nan],
        [nan]], grad_fn=<SliceBackward>)

Esse comportamento é devido a estratégia para automaticamente gerar o gradiente de qualquer estrutura. O PyTorch (e provavelmente outros) transforma o gradiente descendente/backpropagation em um grafo onde cada nó é uma operação. Assim, ele só precisa gerar a derivada de cada nó e aplicar o backpropagation.

### Otimizador

Também podemos selecionar qual tipo de otimizador usaremos para ajustar os parâmetros. O mais comum é o Stochastic Gradient Descent (SGD), mas o PyTorch oferece o Adadelta, Adagrad, RMSProp, entre vários outros dentro do pacoto `torch.optim`. Além disso, claro, também podemos customizar e criar o nosso próprio otimizador.

In [37]:
?torch.optim.RMSprop

Init signature:
torch.optim.RMSprop(
    params,
    lr=0.01,
    alpha=0.99,
    eps=1e-08,
    weight_decay=0,
    momentum=0,
    centered=False,
)
Docstring:     
Implements RMSprop algorithm.

Proposed by G. Hinton in his
`course <https://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf>`_.

The centered version first appears in `Generating Sequences
With Recurrent Neural Networks <https://arxiv.org/pdf/1308.0850v5.pdf>`_.

The implementation here takes the square root of the gradient average before
adding epsilon (note that TensorFlow interchanges these two operations). The effective
learning rate is thus :math:`\alpha/(\sqrt{v} + \epsilon)` where :math:`\alpha`
is the scheduled learning rate and :math:`v` is the weighted moving average
of the squared gradient.

Arguments:
    params (iterable): iterable of parameters to optimize or dicts defining
        parameter groups
    lr (float, optional): learning rate (default: 1e-2)
    momentum (float, optional): momen

Em geral, os otimizadores são variações do Gradient Descent/Backpropagation. Mas alguns são melhores para evitar cair em mínimos locais, ou mesmo para quantidade de épocas necessárias. Outros, como o Adam, podem aumentar a complexidade da otimização ao utilizar valores diferentes para cada parâmetro, consequentemente aumentando também a flexibilidade.

In [46]:
network = torch.nn.Linear(13, 1)
print(network.weight)
print(network.bias)

Parameter containing:
tensor([[ 0.0532, -0.0053, -0.2545,  0.1877, -0.0296, -0.1207,  0.1437, -0.0424,
         -0.1893, -0.2186, -0.0930,  0.0786,  0.0482]], requires_grad=True)
Parameter containing:
tensor([0.0111], requires_grad=True)


In [49]:
norm = StandardScaler()
Xtrain_scl = torch.from_numpy(norm.fit_transform(Xtrain)).type(torch.float)
Xtest_scl = torch.from_numpy(norm.fit_transform(Xtest)).type(torch.float)

pred = network(Xtrain_scl.type(torch.float))
pred[:3]

tensor([[0.0364],
        [0.7296],
        [0.6551]], grad_fn=<SliceBackward>)

In [50]:
mse = torch.nn.functional.mse_loss
mse(pred, Ytrain)

tensor(571.1143, grad_fn=<MseLossBackward>)

In [51]:
optim = torch.optim.SGD(network.parameters(), lr=1e-5)
adam = torch.optim.Adam(network.parameters(), lr=1e-5)

## DataLoader

PyTorch permite encapsular a base de dados para melhor manipular o treinamento e batches da forma mais "pythonica" possível.

In [53]:
tensordata = torch.utils.data.TensorDataset(Xtrain_scl, Ytrain)
tensordata[:2]

(tensor([[-0.4069,  3.5912, -1.4126, -0.2447, -1.2999,  1.0136, -1.8618,  1.8794,
          -0.7319, -0.0153, -0.6847,  0.4268, -1.1580],
         [-0.3192,  0.3843, -1.0419, -0.2447,  0.7997,  1.0682,  0.5845, -0.8064,
          -0.4999, -0.8452, -2.5592,  0.2764,  0.2808]]),
 tensor([[34.9000],
         [30.7000]]))

In [54]:
batch_size = 25
dtl = torch.utils.data.DataLoader(tensordata, batch_size, shuffle=True)

## Treinamento

In [56]:
nepochs = 150
lrate = 1e-5

for epoch in range(nepochs):
    for Xbatch, Ybatch in dtl:
        pred = network(Xbatch)
        loss = mse(pred, Ybatch)
        loss.backward(loss)
        adam.step()
        adam.zero_grad()
    if epoch % 25 == 0:
        print(f'{epoch:3}° {loss}')

  0° 439.104736328125
 25° 358.5230712890625
 50° 513.7679443359375
 75° 338.921875
100° 490.2619934082031
125° 261.1337585449219


## Rede Recorrente

Para redes recorrentes também temos uma classe pronta para ser usada. 

In [57]:
rnn = torch.nn.RNN(input_size=13, hidden_size=13, num_layers=1)

O treinamento (backpropagation through time) é executado automaticamente com a seuinte instrução

In [ ]:
# treina

Com o modelo treinado, podemos realizar nossos testes.

In [ ]:
# testa

Usualmente, as classes do pacote `nn` são tratadas como layers. Ou seja, uma instância da RNN é um layer. Dessa forma, fica fácil gerar modelos complexos e altamente customizáveis. Por exemplo, podemos criar uma Rede Convolucional Recorrente entre outros franksteins.

In [ ]:
# frankstein

### LSTM

Como vimos na parte teórica, a LSTM nada mais é que uma variação do "neurônio" de uma RNN.

![lstmhidden](imgs/lstmhidden.png)

O PyTorch também possui uma classe para o LSTM, tendo em vista sua popularidade.

In [ ]:
# instancia LSTM

Na instância acima, definimos uma LSTM onde o *hidden state* possui dimensão $M\times N\times S$. Uma característica específica da LSTM nesse framework é a entrada esperada e o estado. Ambos devem ser tensores tridimensionais. Podemos treinar a rede ao propagar um elemento da sequência por vez

In [ ]:
# treina 1 em 1

Enquanto que para usar todos os elementos precisamos concatenar nossos dados.

In [ ]:
# todos de uma vez

Principalmente em NLP, a transformação do texto para representação numérica é bastante comum. Dessa forma, podemos criar uma LSTM customizada ao adicionar um layer de embedding que fica responsável por essa transformação.

In [ ]:
# cria classe com embedding layer

Assim como podemos usar o novo layer durante o treinamento, aprendendo os embeddings de acordo com a task.

In [ ]:
# treina embedding + lstm

Claro que também podemos criar vários franksteins, da mesma forma que fizemos com a RNN. Note que o *gradient descent* é totalmente genérico. Isto é, ele não depende da arquitetura do seu modelo, apenas que as funções de ativação sejam deriváveis ou deriváveis por partes. Por isso podemos criar estruturas totalmente **idiotas**, com valores indo, voltando, pulando, ignorando, com muito layer, e com o que quisermos.

In [ ]:
# cria frankstein

É importante notar que redes profundas tendem a ocasionar *overfitting*, assim como os problemas decorrentes do algoritmo de aprendizado:

* Vanishing
* Exploding